In [6]:
from random import randint

for i in range(1,11):
    print(randint(1,10))

1
5
2
3
5
3
9
2
8
8


In [1]:
pc_ref = [0, 25, -60, 400, -20, -75, 0, 300, -62.5, 66.7, 0, 40, -42.9, -25, -66.7, 0, 0, 700, -12.5, -28.6]
pc_tst = [0, 400, -60, 400, -20, -75, 0, 0, 300, 0]

def get_chunks(lst):
    r=[]
    chunk_size = 3
    tot = len(lst)
    for idx, val in enumerate(lst):
        end_idx = idx+chunk_size
        if end_idx <= tot:
            r.append(lst[idx:end_idx])
    return r

ref_chunks = get_chunks(pc_ref)
tst_chunks = get_chunks(pc_tst)

In [3]:
print(ref_chunks)
print(tst_chunks)

[[0, 25, -60], [25, -60, 400], [-60, 400, -20], [400, -20, -75], [-20, -75, 0], [-75, 0, 300], [0, 300, -62.5], [300, -62.5, 66.7], [-62.5, 66.7, 0], [66.7, 0, 40], [0, 40, -42.9], [40, -42.9, -25], [-42.9, -25, -66.7], [-25, -66.7, 0], [-66.7, 0, 0], [0, 0, 700], [0, 700, -12.5], [700, -12.5, -28.6]]
[[0, 400, -60], [400, -60, 400], [-60, 400, -20], [400, -20, -75], [-20, -75, 0], [-75, 0, 0], [0, 0, 300], [0, 300, 0]]


In [4]:
def chunk_diff(ref_lst, tst_lst):
    cntr = 0
    chunk_size = len(tst_lst[0])
    sm_lst = []
    sm_idx_dic = {}
    print(chunk_size)
    for i in tst_lst:
        for j in ref_lst:
            diff = []
            for k in range(chunk_size):
                diff.append(abs(i[k] - j[k]))
            cntr += 1
            sm = sum(diff)
            sm_lst.append({'k': f'{cntr}: {i} vs {j}', 'sm': sm})
    return sm_lst

sm_lst = chunk_diff(ref_chunks, tst_chunks)

3


In [5]:
from typing import List, Dict

def sort_dicts_by_value(dicts: List[Dict], key: str) -> List[Dict]:
    """
    Sorts a list of dictionaries by the specified key value.

    :param dicts: List of dictionaries to be sorted
    :param key: The key by which the dictionaries should be sorted
    :return: A list of dictionaries sorted by the specified key
    """
    try:
        return sorted(dicts, key=lambda x: x[key])
    except KeyError:
        raise KeyError(f"One or more dictionaries do not have the key '{key}'")
    except TypeError:
        raise TypeError(f"The value for key '{key}' is not comparable")

sorted_list = sort_dicts_by_value(sm_lst, 'sm')
print(sorted_list)


[{'k': '39: [-60, 400, -20] vs [-60, 400, -20]', 'sm': 0}, {'k': '58: [400, -20, -75] vs [400, -20, -75]', 'sm': 0}, {'k': '77: [-20, -75, 0] vs [-20, -75, 0]', 'sm': 0}, {'k': '105: [-75, 0, 0] vs [-66.7, 0, 0]', 'sm': 8.299999999999997}, {'k': '86: [-20, -75, 0] vs [-25, -66.7, 0]', 'sm': 13.299999999999997}, {'k': '133: [0, 300, 0] vs [0, 300, -62.5]', 'sm': 62.5}, {'k': '114: [0, 0, 300] vs [-75, 0, 300]', 'sm': 75}, {'k': '99: [-75, 0, 0] vs [-62.5, 66.7, 0]', 'sm': 79.2}, {'k': '3: [0, 400, -60] vs [-60, 400, -20]', 'sm': 100}, {'k': '7: [0, 400, -60] vs [0, 300, -62.5]', 'sm': 102.5}, {'k': '104: [-75, 0, 0] vs [-25, -66.7, 0]', 'sm': 116.7}, {'k': '84: [-20, -75, 0] vs [40, -42.9, -25]', 'sm': 117.1}, {'k': '87: [-20, -75, 0] vs [-66.7, 0, 0]', 'sm': 121.7}, {'k': '103: [-75, 0, 0] vs [-42.9, -25, -66.7]', 'sm': 123.80000000000001}, {'k': '95: [-75, 0, 0] vs [-20, -75, 0]', 'sm': 130}, {'k': '85: [-20, -75, 0] vs [-42.9, -25, -66.7]', 'sm': 139.60000000000002}, {'k': '101: [-75

In [3]:

from binance.client import Client
from binance.enums import *
from time import time
import pickle as pickle
from datetime import datetime
import pandas as pd
from pathlib import Path
from datetime import datetime


downloads_path = str(Path.home() / "Downloads")
tst_fn = f'{downloads_path}\\tst.csv'

def get_client():
    fn = 'C:\\Users\\henok\\OneDrive\\Documents\\key\\binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])


client = get_client()

def get_historical_data(start_timestamp, end_timestamp): 
    data = []
    tot = (end_timestamp - start_timestamp)/(900*500)
    cntr = 0
    for current_sts in range(start_timestamp, end_timestamp+1, 900*500):
        next_ets = current_sts + 900*500 if (current_sts + 900*500) < end_timestamp else end_timestamp
        print(current_sts, next_ets, f'100% completed') if next_ets == end_timestamp else print(current_sts, next_ets, f'{round(cntr*100/tot, 1)}% completed')
        cntr += 1
        klines = client.futures_historical_klines('BTCUSDT', '15m', current_sts*1000, next_ets*1000, limit=500)
        
        for kline in klines:
            timestamp = kline[0]/1000
            open_price = float(kline[1])
            high_price = float(kline[2])
            low_price = float(kline[3])
            close_price = float(kline[4])
            volume = float(kline[5])

            data.append([timestamp, open_price, high_price, low_price, close_price, volume])
    
    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    df.to_csv(tst_fn, index=False)
    print('Data Exported')
    return df

start_timestamp = 1719634904
end_timestamp = 1719721304
hd = get_historical_data(start_timestamp, end_timestamp)

1719634904 1719721304 100% completed
Data Exported


In [2]:
hd

,time,open,high,low,close,volume
0,1.719635e+09,60718.9,60749.2,60706.5,60741.4,678.231
1,1.719636e+09,60741.4,60802.5,60741.3,60775.2,459.453
2,1.719637e+09,60775.2,60788.7,60758.0,60758.1,242.001
3,1.719638e+09,60758.0,60778.2,60723.6,60756.7,484.975
4,1.719639e+09,60756.7,60776.1,60720.4,60723.6,353.103
...,...,...,...,...,...,...
91,1.719717e+09,60827.7,60862.4,60777.5,60832.3,490.053
92,1.719718e+09,60832.3,60937.2,60827.6,60913.9,644.549
93,1.719719e+09,60913.8,60943.6,60879.4,60898.2,551.333
94,1.719720e+09,60898.2,60898.2,60844.9,60867.7,436.889
